In [ ]:

import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text, bindparam
import warnings
warnings.filterwarnings('ignore')

from clabsi_feature_engineering import (
    connect_db, 
    get_clabsi_cohort,
    get_control_cohort,
    CLABSIFeatureProcessor
)

engine = connect_db()
if engine is None:
    raise Exception("Database connection failed. Please check your connection parameters.")

print("Getting cohorts...")
clabsi_cohort = get_clabsi_cohort(engine)
control_cohort = get_control_cohort(engine, clabsi_cohort)


if clabsi_cohort.empty:
    raise Exception("CLABSI cohort is empty!")
if control_cohort.empty:
    raise Exception("Control cohort is empty!")


processor = CLABSIFeatureProcessor(engine)


print("\nProcessing CLABSI cohort...")
clabsi_features = processor.get_batch_features(clabsi_cohort)
if clabsi_features is not None:
    clabsi_features['is_clabsi'] = 1
else:
    raise Exception("Failed to process CLABSI features!")

print("\nProcessing control cohort...")
control_features = processor.get_batch_features(control_cohort)
if control_features is not None:
    control_features['is_clabsi'] = 0
else:
    raise Exception("Failed to process control features!")


combined_df = pd.concat([clabsi_features, control_features], axis=0).reset_index(drop=True)
print(f"\nInitial combined dataset shape: {combined_df.shape}")


print(combined_df.info())
print(combined_df.head())



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import traceback


def load_analysis_data():
    """
    Load analysis data by:
      - Connecting to the MIMIC IV database.
      - Retrieving the CLABSI and control cohorts.
      - Processing features for both cohorts using CLABSIFeatureProcessor.
      - Combining the two datasets.
      
    Returns:
      combined_df (pd.DataFrame): Combined feature dataset.
    """
    try:
        # Connect to database
        engine = connect_db()
        if not engine:
            print("Database connection failed.")
            return None
        
        print("Getting cohorts...")
        clabsi_cohort = get_clabsi_cohort(engine)
        control_cohort = get_control_cohort(engine, clabsi_cohort)
        
        # Initialize feature processor
        processor = CLABSIFeatureProcessor(engine)
        
        # Process CLABSI cohort
        print("Processing CLABSI cohort...")
        clabsi_features = processor.get_batch_features(clabsi_cohort)
        if clabsi_features is not None:
            clabsi_features['is_clabsi'] = 1
        else:
            print("Error processing CLABSI features.")
            return None
        
        # Process control cohort
        print("Processing control cohort...")
        control_features = processor.get_batch_features(control_cohort)
        if control_features is not None:
            control_features['is_clabsi'] = 0
        else:
            print("Error processing control features.")
            return None
        
        # Combine datasets
        combined_df = pd.concat([clabsi_features, control_features], axis=0)
        print(f"\nFinal dataset shape: {combined_df.shape}")
        return combined_df
    
    except Exception as e:
        print(f"Error loading data: {e}")
        traceback.print_exc()
        return None


def clean_combined_data(df):
    """
    Clean and validate the combined dataset.
    
    Steps:
      1. Remove duplicate rows based on 'stay_id'.
      2. Report missing values in critical columns.
      3. For both CLABSI cases and controls, impute negative 
         'days_since_last_dressing_change' values with 0.
      4. Validate that key numeric values fall within reasonable ranges.
      5. Report the number of rows removed and the final CLABSI distribution.
    """
    import warnings

    print("\nStarting data cleaning process...")
    df_clean = df.copy()
    original_shape = df_clean.shape

    # Remove duplicates based on 'stay_id'
    duplicates = df_clean.duplicated(subset=['stay_id'], keep='first')
    if duplicates.any():
        num_duplicates = duplicates.sum()
        print(f"Found {num_duplicates} duplicate stay_ids. Removing duplicates.")
        df_clean = df_clean.drop_duplicates(subset=['stay_id'], keep='first')
    else:
        print("No duplicate stay_ids found.")

    # Define critical columns for cleaning diagnostics
    critical_columns = [
        'total_baths', 'days_since_last_dressing_change',
        'admission_age', 'plt_mean', 'wbc_mean'
    ]
    missing_overall = df_clean[critical_columns].isnull().sum()
    print("\nMissing values in critical columns (overall):")
    print(missing_overall[missing_overall > 0])
    
    # Handle negative values in 'days_since_last_dressing_change'
    if 'is_clabsi' in df_clean.columns:
        cases = df_clean[df_clean['is_clabsi'] == 1].copy()
        controls = df_clean[df_clean['is_clabsi'] == 0].copy()
        
        neg_cases = (cases['days_since_last_dressing_change'] < 0).sum()
        neg_controls = (controls['days_since_last_dressing_change'] < 0).sum()
        if neg_cases > 0:
            print(f"Found {neg_cases} negative 'days_since_last_dressing_change' values in cases. Imputing with 0.")
            cases.loc[cases['days_since_last_dressing_change'] < 0, 'days_since_last_dressing_change'] = 0
        if neg_controls > 0:
            print(f"Found {neg_controls} negative 'days_since_last_dressing_change' values in controls. Imputing with 0.")
            controls.loc[controls['days_since_last_dressing_change'] < 0, 'days_since_last_dressing_change'] = 0
        
        df_clean = pd.concat([cases, controls], axis=0)
    else:
        neg_count = (df_clean['days_since_last_dressing_change'] < 0).sum()
        if neg_count > 0:
            print(f"Found {neg_count} negative 'days_since_last_dressing_change' values. Imputing with 0.")
            df_clean.loc[df_clean['days_since_last_dressing_change'] < 0, 'days_since_last_dressing_change'] = 0


    print("\nValidating value ranges...")
    df_clean = df_clean[(df_clean['admission_age'] > 0) & (df_clean['admission_age'] < 120)]
    df_clean = df_clean[(df_clean['plt_mean'] > 0) & (df_clean['wbc_mean'] > 0)]
    
    final_shape = df_clean.shape
    rows_removed = original_shape[0] - final_shape[0]
    print(f"\nOriginal shape: {original_shape}")
    print(f"Cleaned shape: {final_shape}")
    print(f"Removed {rows_removed} rows ({rows_removed/original_shape[0]:.1%}) during cleaning")
    
    if 'is_clabsi' in df_clean.columns:
        clabsi_dist = df_clean['is_clabsi'].value_counts(normalize=True)
        print("\nCLABSI distribution after cleaning:")
        print(clabsi_dist)
    
    return df_clean

def prepare_features(combined_df):
    """
    Prepare features for importance analysis from the cleaned dataset.
    
    Returns:
      X (pd.DataFrame): Feature matrix.
      y (pd.Series): Target variable.
      feature_categories (dict): Dictionary of feature categories.
    """
    # Define feature categories (note: 'total_baths' has been removed)
    feature_categories = {
        'line_characteristics': [
            'line_duration_days', 'line_type', 'line_site', 
            'site_category', 'risk_category'
        ],
        'patient_factors': [
            'admission_age', 'gender', 'has_diabetes', 'has_cancer',
            'has_liver', 'has_chf', 'has_cva'
        ],
        'clinical_care': [
            'days_since_last_dressing_change', 'chg_adherence_ratio'
        ],
        'lab_values': [
            'wbc_mean', 'plt_mean', 'creat_mean', 'inr_mean',
            'pt_mean', 'ptt_mean'
        ],
        'severity_scores': [
            'sofa_score', 'apsiii', 'sapsii'
        ]
    }
    
    
    y = combined_df['is_clabsi']
    
   
    feature_cols = []
    for cat in feature_categories.values():
        feature_cols.extend([col for col in cat if col in combined_df.columns])
    
  
    X = combined_df[feature_cols].copy()
    
   
    categorical_features = X.select_dtypes(include=['object']).columns
    X = pd.get_dummies(X, columns=categorical_features)
    
   
    X = X.fillna(X.mean())
    
    return X, y, feature_categories


def calculate_feature_importance(X, y):
    """
    Calculate feature importance using:
      1. XGBoost built-in feature importance.
      2. Mutual Information.
      3. SHAP values.
      
    Returns:
      importance_df (pd.DataFrame): DataFrame with composite importance scores.
    """
    from sklearn.model_selection import train_test_split
    from sklearn.feature_selection import SelectKBest, mutual_info_classif
    import xgboost as xgb
    import shap
    
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
   
    xgb_model = xgb.XGBClassifier(
        random_state=42,
        n_estimators=100,
        learning_rate=0.1
    )
    xgb_model.fit(X_train, y_train)
    xgb_importance = pd.DataFrame({
        'feature': X.columns,
        'xgb_importance': xgb_model.feature_importances_
    })
    
    
    mi_selector = SelectKBest(score_func=mutual_info_classif, k='all')
    mi_selector.fit(X_train, y_train)
    mi_importance = pd.DataFrame({
        'feature': X.columns,
        'mutual_info_score': mi_selector.scores_
    })
    
   
    explainer = shap.TreeExplainer(xgb_model)
    shap_values = explainer.shap_values(X_test)
    shap_importance = pd.DataFrame({
        'feature': X.columns,
        'shap_importance': np.abs(shap_values).mean(axis=0)
    })
    
   
    importance_df = xgb_importance.merge(mi_importance, on='feature')
    importance_df = importance_df.merge(shap_importance, on='feature')
    
   
    for col in ['xgb_importance', 'mutual_info_score', 'shap_importance']:
        importance_df[col] = importance_df[col] / importance_df[col].max()
    
    
    importance_df['composite_score'] = importance_df[['xgb_importance', 'mutual_info_score', 'shap_importance']].mean(axis=1)
    
    return importance_df.sort_values('composite_score', ascending=False)


def diagnostic_plots(df, group_col='is_clabsi', critical_columns=None, title_prefix="Before Cleaning"):
    """
    Plot summary statistics and histograms for key critical columns,
    optionally grouped by a specified column.
    """
    if critical_columns is None:
        critical_columns = ['total_baths', 'days_since_last_dressing_change', 'admission_age', 'plt_mean', 'wbc_mean']
    
    if group_col in df.columns:
        groups = df.groupby(group_col)
    else:
        groups = [("All", df)]
    
    for group_name, group_df in groups:
        print(f"\nSummary statistics for group '{group_name}' {title_prefix}:")
        print(group_df[critical_columns].describe())
        print(f"\nMissing values for group '{group_name}' {title_prefix}:")
        print(group_df[critical_columns].isnull().sum())
        for col in critical_columns:
            plt.figure(figsize=(6,4))
            plt.hist(group_df[col].dropna(), bins=20, edgecolor='black')
            plt.title(f"{title_prefix}: Histogram of {col} (Group: {group_name})")
            plt.xlabel(col)
            plt.ylabel("Frequency")
            plt.show()


if __name__ == "__main__":
    print("Loading and preparing data...")
    
    
    combined_df = load_analysis_data()
    
    if combined_df is not None:
        # Optional: Diagnostics before cleaning
        print("\n--- DIAGNOSTICS BEFORE CLEANING ---")
        print("\nOverall summary statistics (critical columns):")
        print(combined_df[['total_baths', 'days_since_last_dressing_change', 'admission_age', 'plt_mean', 'wbc_mean']].describe())
        print("\nOverall missing values (critical columns):")
        print(combined_df[['total_baths', 'days_since_last_dressing_change', 'admission_age', 'plt_mean', 'wbc_mean']].isnull().sum())
        diagnostic_plots(combined_df, group_col='is_clabsi', title_prefix="Before Cleaning")
        
        
        print("\nCleaning data...")
        cleaned_df = clean_combined_data(combined_df)
        
       
        print("\n--- DIAGNOSTICS AFTER CLEANING ---")
        print("\nOverall summary statistics (critical columns):")
        print(cleaned_df[['total_baths', 'days_since_last_dressing_change', 'admission_age', 'plt_mean', 'wbc_mean']].describe())
        print("\nOverall missing values (critical columns):")
        print(cleaned_df[['total_baths', 'days_since_last_dressing_change', 'admission_age', 'plt_mean', 'wbc_mean']].isnull().sum())
        diagnostic_plots(cleaned_df, group_col='is_clabsi', title_prefix="After Cleaning")
        
        
        print("\nPreparing features from cleaned data...")
        X, y, feature_categories = prepare_features(cleaned_df)
        
        
        print("\nCalculating feature importance...")
        importance_df = calculate_feature_importance(X, y)
        
        # Save the importance results to CSV and display top 20 features
        importance_df.to_csv('feature_importance_results.csv', index=False)
        print("\nTop 20 Most Important Features:")
        print(importance_df.head(20))
        
       
        def plot_feature_importance(importance_df, top_n=20):
            top_features = importance_df.sort_values('composite_score', ascending=False).head(top_n)
            plt.figure(figsize=(10, 6))
            plt.barh(top_features['feature'], top_features['composite_score'], color='skyblue')
            plt.xlabel('Composite Importance Score (Normalized)')
            plt.title(f'Top {top_n} Most Important Features')
            plt.gca().invert_yaxis()
            plt.show()
        
        plot_feature_importance(importance_df, top_n=20)
    else:
        print("Error: Could not load or prepare data properly")



In [ ]:
import matplotlib.pyplot as plt

def plot_feature_importance(importance_df, top_n=20):
    
    top_features = importance_df.sort_values('composite_score', ascending=False).head(top_n)
    
    
    plt.figure(figsize=(10, 6))
    plt.barh(top_features['feature'], top_features['composite_score'], color='skyblue')
    plt.xlabel('Composite Importance Score (Normalized)')
    plt.title(f'Top {top_n} Most Important Features')
    plt.gca().invert_yaxis()  # Highest scores at the top
    plt.show()

# Plot the top 20 important features
plot_feature_importance(importance_df, top_n=20)


def select_top_features(importance_df, threshold=0.2):
    """
    Select features whose composite importance score exceeds a threshold.
    """
    # For example, select features with a composite score greater than the threshold.
    selected_features = importance_df[importance_df['composite_score'] >= threshold]['feature'].tolist()
    print(f"Selected {len(selected_features)} features with composite score >= {threshold}:")
    print(selected_features)
    return selected_features


selected_features = select_top_features(importance_df, threshold=0.2)


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb


X_final = X[selected_features]
y_final = y


X_train, X_test, y_train, y_test = train_test_split(
    X_final, y_final, test_size=0.2, random_state=42
)

# Train a final XGBoost model
final_model = xgb.XGBClassifier(random_state=42, n_estimators=100, learning_rate=0.1)
final_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = final_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nFinal model accuracy with selected features: {accuracy:.4f}")


In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
import joblib
import traceback


best_params = {
    'colsample_bytree': 0.8,
    'learning_rate': 0.1,
    'max_depth': 7,
    'n_estimators': 200,
    'subsample': 1.0
}



print("Splitting data for model evaluation...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instantiate the XGBoost classifier with the best parameters
eval_model = xgb.XGBClassifier(
    colsample_bytree=best_params['colsample_bytree'],
    learning_rate=best_params['learning_rate'],
    max_depth=best_params['max_depth'],
    n_estimators=best_params['n_estimators'],
    subsample=best_params['subsample'],
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)

print("Training model on the training set...")
eval_model.fit(X_train, y_train)


y_pred = eval_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, eval_model.predict_proba(X_test)[:, 1])
print("\nEvaluation Metrics on Test Data:")
print("Test Accuracy: {:.4f}".format(accuracy))
print("Test ROC AUC: {:.4f}".format(roc_auc))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


print("\nRetraining final model on the full dataset...")
final_model = xgb.XGBClassifier(
    colsample_bytree=best_params['colsample_bytree'],
    learning_rate=best_params['learning_rate'],
    max_depth=best_params['max_depth'],
    n_estimators=best_params['n_estimators'],
    subsample=best_params['subsample'],
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)

try:
    # Train on the full dataset
    final_model.fit(X, y)
    
    # Save the final model to disk (you can choose either joblib or xgboost's save_model)
    joblib.dump(final_model, 'final_xgb_model.pkl')
    print("Final model has been retrained on the full dataset and saved as 'final_xgb_model.pkl'.")
except Exception as e:
    print(f"Error during final model training or saving: {e}")
    traceback.print_exc()


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
import xgboost as xgb
import joblib


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instantiate the final model with the best hyperparameters you found:
# {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'subsample': 1.0}
final_model = xgb.XGBClassifier(
    colsample_bytree=0.8,
    learning_rate=0.1,
    max_depth=7,
    n_estimators=200,
    subsample=1.0,
    random_state=42,
    use_label_encoder=False,  # Suppress deprecation warnings in newer versions
    eval_metric='logloss'
)

# Train the model on the training set
final_model.fit(X_train, y_train)

# Evaluate on the test set
y_pred = final_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, final_model.predict_proba(X_test)[:, 1])

print("Final Model Test Accuracy: {:.4f}".format(accuracy))
print("Final Model Test ROC AUC: {:.4f}".format(roc_auc))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
